# Kuramoto-Sivashinky Equation Model


In [1]:
using FFTW
using LaTeXStrings
using LinearAlgebra
using Plots
using SparseArrays

include("../src/model/KS.jl")
include("../src/LiftAndLearn.jl")
const LnL = LiftAndLearn
;


## Kuramoto-Sivashinsky (KS) Equation
The Kuramoto-Sivashinsky (KS) equation is a partial differential equation that describes the dynamics of certain spatiotemporal patterns in various physical systems, particularly in fluid dynamics and combustion processes. It was introduced by Yoshiki Kuramoto and Grigoriĭ Isaakovich Sivashinsky independently in the 1970s.

The equation is given by:

$$
\begin{gather*}
    u_t + uu_x + u_{xx} + \mu u_{xxxx} = 0,  \qquad x \in (-\infty, \infty) \\
    u(x,t) = u(x+L,t), \quad u_x(x,t) = u_x(x+L,t), \quad t \geq 0 
\end{gather*}
$$

where:
- $u(x, t)$ represents the spatially and temporally varying quantity being modeled (e.g., the amplitude of certain patterns in a fluid or combustion system).
- $t$ is time.
- $x$ is a spatial coordinate.

The terms in the equation have the following meanings:
- $u_t$: Represents the time rate of change of the quantity u.
- $uu_x$: Represents advection, describing how the quantity u is transported along the spatial coordinate x by its own gradient.
- $u_{xx}$: Represents diffusion, describing how the quantity u diffuses or spreads out over space.
- $u_{xxxx}$: Represents a fourth-order spatial derivative that accounts for certain nonlinear interactions and dispersion effects.
- $\mu$: Viscosity parameter.

The Kuramoto-Sivashinsky equation is known for producing a rich variety of complex spatiotemporal patterns, including traveling waves, oscillations, and turbulence-like behavior. It has applications in understanding and modeling various physical phenomena, such as the dynamics of flames, fluid interfaces, and certain chemical reactions. The equation's complexity and the diversity of patterns it can exhibit make it a subject of study in nonlinear dynamics and chaos theory.

For our analysis, we will construct a numerical model by descretizing the original PDE and separating the system into the linear and nonlinear components in the form of 

$$
\begin{gather*}
    \dot{\mathbf u} = \mathbf A \mathbf u + \mathbf F \mathbf u^{\langle 2\rangle}
\end{gather*}
$$

where $\mathbf A$, $\mathbf F$ are the linear and nonlinear operators respectively.


## Numerical Integration

Once we have the numerical model, we integrate it using the Adam-Bashforth and Crank-Nicolson Implicit scheme. The method is as follows.

$$
\begin{gather*}
    \dot{\mathbf u} = \mathbf A \mathbf u + \mathbf F \mathbf u^{\langle 2\rangle} \\
    \frac{\mathbf u^{k+1} - \mathbf u^k}{\Delta t} = \mathbf A \left(\frac{\mathbf u^{k+1} + \mathbf u^k}{2}\right) + \left[ \frac{3}{2}\mathbf F \left( \mathbf u^k \right)^{\langle 2\rangle} -\frac{1}{2}\mathbf F\left( u^{k-1} \right)^{\langle 2 \rangle } \right] 
\end{gather*}
$$

Hence 

$$
\begin{gather*}
    \mathbf u^{k+1} = 
    \begin{cases} 
        \left(\mathbf I - \frac{\Delta t}{2}\mathbf A \right)^{-1}\left[ \left( \mathbf I + \frac{\Delta t}{2}\mathbf A \right)\mathbf u^k + \Delta t\mathbf F \left( \mathbf u^k \right)^{\langle 2\rangle} \right] \qquad  k = 1 \\[0.3cm]
        \left(\mathbf I - \frac{\Delta t}{2}\mathbf A \right)^{-1}\left[ \left( \mathbf I + \frac{\Delta t}{2}\mathbf A \right)\mathbf u^k + \frac{3\Delta t}{2}\mathbf F \left( \mathbf u^k \right)^{\langle 2\rangle} -\frac{\Delta t}{2}\mathbf F\left( u^{k-1} \right)^{\langle 2 \rangle }\right] \qquad  k \geq 2
    \end{cases}
\end{gather*}
$$


## Numerical Model


In [2]:
# Settings for the KS equation
KSE = KS(
    [0.0, 25.0], [0.0, 200.0], [1.0, 1.0],
    512, 0.001, 1, "nc"
)

DS = 100
L = KSE.Omega[2]

# Initial condition
u0 = cos.((2*π*KSE.x)/L) + 0.1*cos.((4*π*KSE.x)/L)  # initial condition version 1
# u0 = cos.(2*π*KSE.x / L) .* (1 .+ sin.(2*π*KSE.x / L))  # initial condition version 2
;

### Finite Difference Method

Similar to the discretization of the [1D Heat equation](01_heat1D_OpInf_LS.ipynb), [1D Burgers' equation](02_burgers_OpInf_LS.ipynb), and [Fitzhugh-Nagumo equation](03_fhn_LnL_LS.ipynb) we discretize the PDE using finite difference approach. In order to discretize in the spatial and temporal domains we definte the grid size $\Delta x$ and time-step $\Delta t$. Furthermore, let the subscript and superscript indicate the spatial and temporal indices respectively, i.e., $u_{i}^k$. Then we show how we discretize each term below.

$$
\begin{align*}
    u_{xxxx} &\approx \frac{1}{\Delta x^4} \left( u_{i+2} - 4u_{i+1} + 6u_i - 4u_{i-1} + u_{i-2} \right) \\
    u_{xx} &\approx \frac{1}{\Delta x^2} \left( u_{i+1} - 2u_i + u_{i-1} \right) \\
    u_x &\approx \frac{1}{2\Delta x} \left( u_{i+1} - u_{i-1} \right) \quad .
\end{align*}
$$

Then we can represent the KS equation model with distinct linear and nonlinear terms

$$
\begin{gather*}
    \dot u_i = \underbrace{ \left[ -\frac{\mu}{\Delta x^4} (u_{i+2} + u_{i-2}) + \left( \frac{4\mu}{\Delta x^4}-\frac{1}{\Delta x^2} \right)(u_{i+1} + u_{i-1}) + \left( \frac{2}{\Delta x^2}-\frac{6\mu}{\Delta x^4} \right)u_i \right] }_{\text{linear}}
    + \underbrace{\frac{1}{2\Delta x}\left( u_iu_{i-1} - u_iu_{i+1} \right)}_{\text{nonlinear}}
\end{gather*}
$$

Thus, assuming we have periodic boundary conditions, we can represent the KS equation as a linear-quadratic ODE with respect to time by expanding the above expression to all of the spatial grid:

$$
\begin{gather*}
    \mathbf{\dot u}(t) = \mathbf{A}\mathbf u(t) + \mathbf{F} \mathbf u^{\langle 2 \rangle}(t)
\end{gather*}
$$

where $\mathbf A \in \mathbb R^{N\times N}$ is the linear operator, $\mathbf F \in \mathbb R^{N \times N(N+1)/2}$ is the quadratic operator, and  $\mathbf u^{\langle 2\rangle} \in \mathbb R^{N(N+1)/2}$ represents the quadratic states with no redundancy. The matrix $\mathbf A$ would be a toeplitz matrix (except for the periodic terms) and the $\mathbf F$ would be a sparse matrix.


In [ ]:
# Generate the operators
A, F = KSE.model_FD(KSE, KSE.μs[1])
;

In [ ]:
# Integrate
u_FD = KSE.integrate_FD(A, F, KSE.t, u0; const_stepsize=true)
;

In [ ]:
plot(contourf(KSE.t[1:100:end], KSE.x, u_FD[:, 1:100:end], lw=0), xlabel=L"t, \mathrm{time}",
    ylabel=L"x, \mathrm{space}", zlabel=L"u(x,t)", size=(680, 400))
plot!(guidefontsize=16, tickfontsize=8,  legendfontsize=13)

In [ ]:
p1 = plot(fontfamily="Computer Modern")
for i in 1:1000:KSE.Tdim
    plot!(p1, KSE.x, u_FD[:, i])
end
plot!(p1, xlabel=L"x", ylabel=L"u(x,t)", grid=true,
    minorgrid=true, legend=false)
plot!(guidefontsize=16, tickfontsize=8,  legendfontsize=13)

### Spectral Method

We consider a periodic domain $[0,L]$ for the solution of the Kuramoto-Sivashinsky equation. Our periodic grid has $N$ points $\{x_n\}_{n=1}^N$, where $x_n = n\Delta x$ and $\Delta x = L/N$. With $N$ degrees of freedom we can write the solution $u(x,t)$ as a truncated Fourier expansion with $N$ modes:
$$
\begin{align*}
u(x,t) = \int_{-\infty}^\infty \hat u_k(t)\exp\left(\frac{2\pi j kx}{L}\right)dk \approx \sum_{k=-N/2}^{N/2-1}\hat{u}_k(t)\exp\left(\frac{2\pi jkx}{L}\right).
\end{align*}
$$

Now from here, we introduce two possible methods using the Fourier transform. The first method, uses the Fast Fourier Transform (FFT) or Pseudo-Spectral (PS) method to deal with the nonlinear term. In contrast, the second method directly uses the Fourier Mode by formulating the problem with the Spectral Galerkin (SG) method.

#### Pseudo-Spectral Method

We begin by plugging the approximate Fourier transform of $u(x,t)$ back into the original PDE which give us 

$$
\begin{align*}
    \dot u(x,t) &\approx \sum_{k=-N/2}^{N/2-1} \dot{\hat u_k}(t)\exp\left( \frac{2\pi jkx}{L} \right) \\
    \mu u_{xxxx} + u_{xx} &\approx  \sum_{k=-N/2}^{N/2-1}\mu\left( \frac{2\pi j k}{L} \right)^4 \hat u_k(t) \exp\left( \frac{2\pi j k x}{L} \right) + \sum_{k=-N/2}^{N/2-1}\left( \frac{2\pi j k}{L} \right)^2  \hat u_k(t) \exp\left( \frac{2\pi j kx}{L} \right) = -\sum_{k=-N/2}^{N/2-1} \left[ \left( \frac{2\pi k}{L} \right)^2 - \mu\left( \frac{2\pi  k}{L} \right)^4 \right] \hat u_k(t) \exp\left( \frac{2\pi j kx}{L} \right) \\
    uu_x &= \frac{1}{2}\left( u^2 \right)_x \approx \frac{1}{2} \frac{2\pi j k}{L}\left( \hat u^2 \right)_k = \frac{1}{2} \frac{2\pi j k }{L} ~\left(\mathrm{FFT}\left[ u^2(t) \right]\right)_k
\end{align*}
$$

Then if we collect the terms within the summation and multiplied by $\exp(2\pi jk x/L )$ we have

$$
\begin{gather*}
    \dot{\hat u}_k(t) = \underbrace{\left[ \left( \frac{2\pi k}{L}\right)^2 - \mu\left( \frac{2\pi k}{L} \right)^4 \right]\hat u_k(t)}_{\text{linear}} - \underbrace{\frac{\pi j k}{L}~\left(\mathrm{FFT}\left[ u^2(t) \right]\right)_k}_{\text{nonlinear}}
\end{gather*}
$$

For more detail on the derivation, refer to [this paper](http://pubs.sciepub.com/ajna/2/3/5/abstract.html) by Gentian Zavalani. If write this in the form of $\mathbf{\dot u}(t) = \mathbf{A}\mathbf u(t) + \mathbf{F} \mathbf u^{\langle 2 \rangle}(t)$ we will have a diagonal matrix of $\mathbf A$ and for $\mathbf F$. The original states $u(t)$ of the KS equation can be retained by performing the inverse FFT (iFFT) on the states $\hat u(t)$.

Or you could let $\mathbf A$ and $\mathbf F$ be a vector and do element-wise multiplications to speed-up the integration process.

In [ ]:
# Generate the operators
A, F = KSE.model_PS(KSE, KSE.μs[1])
;

In [ ]:
# Integrate
u_PS, _ = KSE.integrate_PS(A, F, KSE.t, u0)
;

In [ ]:
plot(contourf(KSE.t[1:DS:end], KSE.x, u_PS[:, 1:DS:end], lw=0), xlabel=L"t, \mathrm{time}",
    ylabel=L"x, \mathrm{space}", zlabel=L"u(x,t)", size=(680, 400))
plot!(guidefontsize=16, tickfontsize=8,  legendfontsize=13)

In [ ]:
p1 = plot()
for i in 1:1000:KSE.Tdim
    plot!(p1, KSE.x, u_PS[:, i])
end
plot!(p1, xlabel=L"x", ylabel=L"u(x,t)", grid=true,
    minorgrid=true, legend=false)
plot!(guidefontsize=16, tickfontsize=8,  legendfontsize=13)

#### Spectral Galerkin (SP) Method

In the SP method, you take the inner product between the Fourier transformed expression and the exponential to retrieve the Fourier coefficient with the orthogonality condition. For example, for the $-u_{xx}$ term

$$
\begin{align*}
    \left\langle -\partial_x u, \partial_x u\right\rangle &= \left\langle -\partial_x \sum_{l=-N/2}^{N/2-1}\hat u_l(t) \exp\left(\frac{2\pi j l}{L}x\right), ~\partial_x\exp\left( \frac{2\pi j k}{L}x\right) \right\rangle \\
    &= \left\langle - \sum_{l=-N/2}^{N/2-1} \frac{2\pi j l}{L} \hat u_l(t) \exp\left(\frac{2\pi j l}{L}x\right), ~ \frac{2\pi jk}{L} \exp\left( \frac{2\pi j k}{L}x\right) \right\rangle \\
    &{\xrightarrow{l=k}} \left( \frac{2\pi k }{L} \right)^2 \hat u_k(t)
\end{align*}
$$

For $u_{xxxx}$ we obtain the same expression as the previous pseudo-spectral method, and therefore, the linear part of the model is exactly the same. However, we take a different route for the nonlinear term. If we consider the conservative advection nonlinearity, the spectral Galerkin would be as follows. 

$$
\begin{align*}
    \left\langle -\frac{1}{2}u^2, ~\partial_x\exp\left( \frac{2\pi jk}{L}x\right)\right\rangle &= \left\langle  -\frac{1}{2}\left[\sum_{p=-N/2}^{N/2-1}\hat u_p(t) \exp\left(\frac{2\pi j p}{L}x\right)\right]\left[\sum_{q=-N/2}^{N/2-1}\hat u_q(t) \exp\left(\frac{2\pi j q}{L}x\right) \right], ~\frac{2\pi jk}{L}\exp\left( \frac{2\pi jk}{L} \right) \right\rangle \\
    &=\left\langle  -\frac{1}{2} \sum_{p=-N/2}^{N/2-1}\sum_{q=-N/2}^{N/2-1}\hat u_p(t) \hat u_q(t) \exp\left[\frac{2\pi j (p+q)}{L}x\right], ~\frac{2\pi jk}{L}\exp\left( \frac{2\pi jk}{L} \right) \right\rangle \\
    &\xrightarrow{p+q~=~k} -\frac{\pi jk}{L}\sum_{p+q=k} \hat u_p(t) \hat u_q(t)
\end{align*}
$$


<!-- $$
\begin{align*}
    uu_x & \approx \left[ \sum_{k=-N/2}^{N/2-1} \hat u_k(t) \exp\left( \frac{2\pi jkx}{L} \right) \right] \left[ \sum_{l=-N/2}^{N/2-1} \left( \frac{2\pi j l}{L} \right) \hat u_l(t) \exp\left( \frac{2\pi jlx}{L} \right) \right] \\
    &=  \sum_{k,l=-N/2}^{N/2-1} \left( \frac{2\pi j l}{L} \right) \hat u_k(t) \hat u_l(t) \exp\left( \frac{2\pi j(k+l)x}{L} \right)   \\
\end{align*}
$$

Let $k = m$ and $l = k - m$, then

$$
\begin{align*}
    uu_x &\approx  \sum_{m,k-m=-N/2}^{N/2-1} \frac{2\pi j(k-m)}{L} \hat u_m(t) \hat u_{k-m}(t) \exp\left( \frac{2\pi jkx}{L} \right) 
\end{align*}
$$ -->

With the linear and nonlinear terms together, we have

$$
\begin{gather*}
    \dot{\hat u}_k(t) = \underbrace{\left[ \left( \frac{2\pi k}{L}\right)^2 - \mu\left( \frac{2\pi k}{L} \right)^4 \right]}_{\text{linear}}\hat u_k(t) +  \underbrace{\frac{-\pi jk}{L} \sum_{p+q=k}\hat u_p(t) \hat u_q(t)}_{\text{nonlinear}} ~ .
\end{gather*}
$$

<!-- To change the index $k-m$, we first let $p=k-m$ and then $p \rightarrow m$, which leads to the following expression:

$$
\begin{gather*}
    \dot{\hat u}_k(t) = \underbrace{\left[ \left( \frac{2\pi k}{L}\right)^2 - \mu\left( \frac{2\pi k}{L} \right)^4 \right]\hat u_k(t)}_{\text{linear}} + \underbrace{\sum_{m=-N/2}^{N/2-1} \left(-\frac{2\pi jm}{L}\right)  \hat u_m(t) \hat u_{k-m}(t)}_{\text{nonlinear}} \quad .
\end{gather*}
$$ -->

Assume $\hat u_k$ is pure imaginary. Then let us define  $\hat u_k(t) = j\hat v_k(t)$, where $v_k(t) \in \mathbb R^N$, to limit the model in the real space, which brings us to

$$
\begin{gather*}
    \dot{\hat v}_k(t) = \underbrace{\left[ \left( \frac{2\pi k}{L}\right)^2 - \mu\left( \frac{2\pi k}{L} \right)^4 \right]\hat v_k(t)}_{\text{linear}} + \underbrace{\frac{-\pi k}{L} \sum_{p+q=k}\hat v_p(t) \hat v_q(t)}_{\text{nonlinear}} \quad .
\end{gather*}
$$

Thus, in the real Fourier space, the model can be expressed as 

$$
\begin{gather*}
    \dot{\hat{\mathbf{v}}}(t) = \mathbf{A}\hat{\mathbf v}(t) + \mathbf{F} \hat{\mathbf v}^{\langle 2 \rangle}(t) \quad .
\end{gather*}
$$

Now, since the Fourier transform $\mathcal F$ is a linear operator, so is the inverse Fourier transform $\mathcal F^{-1}$. Hence, the KS model in the time domain is expressed as 

$$
\begin{gather*}
    \mathcal F^{-1}\left[ \dot{\hat{\mathbf{v}}}(t) \right] = \mathbf{A}~\mathcal F^{-1} \left[\hat{\mathbf v}(t)\right] + \mathbf{F} ~\mathcal F^{-1}\left[\hat{\mathbf v}^{\langle 2 \rangle}(t) \right] \quad .
\end{gather*}
$$

where the linear and quadratic operators do not change. For more details consult [this paper](https://dx.doi.org/10.1088/0951-7715/10/1/004). 

However if we would not want to assume $\hat u_k$ to be pure imaginary then we will just let 

$$
\begin{gather*}
    \dot{\hat{\mathbf{u}}}(t) = \mathbf{A}\hat{\mathbf u}(t) + \mathbf{F} \hat{\mathbf u}^{\langle 2 \rangle}(t) \quad ,
\end{gather*}
$$

where $\mathbf F$ would have complex valued entries. To acquire the original states $u(t)$ of the KS equation we will have to perform the inverse FFT (iFFT) on the state data in the Fourier space.


In [ ]:
# Generate the operators
A, F = KSE.model_SG(KSE, KSE.μs[1])
;

In [ ]:
# Integrate
u_SG, _ = KSE.integrate_SG(A, F, KSE.t, u0)
;

In [ ]:
plot(contourf(KSE.t, KSE.x, u_SG, lw=0), xlabel=L"t, \mathrm{time}",
    ylabel=L"x, \mathrm{space}", zlabel=L"u(x,t)", size=(680, 400))
plot!(guidefontsize=16, tickfontsize=8,  legendfontsize=13)

In [ ]:
p1 = plot()
for i in 1:1000:KSE.Tdim
    plot!(p1, KSE.x, u_SG[:, i])
end
plot!(p1, xlabel=L"x", ylabel=L"u(x,t)", grid=true,
    minorgrid=true, legend=false)
plot!(guidefontsize=16, tickfontsize=8,  legendfontsize=13)

## Animation

In [ ]:
@gif for i ∈ 2:KSE.Tdim
    plot(KSE.x, u_FD[:,i], xlabel="x", ylabel="u(x,t)", fontfamily="Modern Computer", label="")
    plot!(KSE.x, u_FD[:,1], label="")
    ylims!(-4,4)
end every 10